# 3. Advanced Image Analysis

**Project:** Image EDA (Olivetti Faces)  
**Goal:** Apply advanced image analysis techniques including color histograms, edge detection, feature extraction, and compare multiple dimensionality reduction methods.

---

## 1. Imports and Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_olivetti_faces
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap
import os

# Try importing optional dependencies
try:
    import cv2
    CV2_AVAILABLE = True
except ImportError:
    print("OpenCV not available. Install with: pip install opencv-python")
    CV2_AVAILABLE = False

try:
    from skimage import feature, color
    from skimage.feature import hog
    SKIMAGE_AVAILABLE = True
except ImportError:
    print("scikit-image not available. Install with: pip install scikit-image")
    SKIMAGE_AVAILABLE = False

try:
    import umap
    UMAP_AVAILABLE = True
except ImportError:
    print("UMAP not available. Install with: pip install umap-learn")
    UMAP_AVAILABLE = False

sns.set_style('white')
plt.rcParams['figure.figsize'] = (12, 6)

## 2. Load Dataset

In [ ]:
print("Loading dataset...")
data_home = '../../data/raw/scikit_learn_data'
if not os.path.exists(data_home):
    os.makedirs(data_home)

olivetti = fetch_olivetti_faces(shuffle=True, random_state=42, data_home=data_home)

X = olivetti.data
y = olivetti.target
n_samples, n_features = X.shape
h, w = 64, 64

print(f"Dataset Shape: {X.shape}")
print(f"Image dimensions: {h}x{w}")
print(f"Number of people: {len(np.unique(y))}")

## 3. Color Histogram Analysis
Analyze the distribution of pixel intensities (grayscale histogram for this dataset).

In [ ]:
# Select a sample image
sample_img = X[0].reshape(h, w)

# Create histogram
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Show image
axes[0].imshow(sample_img, cmap='gray')
axes[0].set_title('Sample Image')
axes[0].axis('off')

# Show histogram
axes[1].hist(sample_img.ravel(), bins=50, color='gray', alpha=0.7)
axes[1].set_title('Pixel Intensity Histogram')
axes[1].set_xlabel('Pixel Value (0-1 scaled)')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print(f"Mean intensity: {sample_img.mean():.3f}")
print(f"Std deviation: {sample_img.std():.3f}")
print(f"Min value: {sample_img.min():.3f}")
print(f"Max value: {sample_img.max():.3f}")

In [ ]:
# Compare histograms across multiple images
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for idx in range(8):
    img = X[idx * 50].reshape(h, w)  # Sample every 50th image
    row = idx // 4
    col = idx % 4
    
    axes[row, col].hist(img.ravel(), bins=30, color='steelblue', alpha=0.7)
    axes[row, col].set_title(f'Person {y[idx * 50]}')
    axes[row, col].set_xlim(0, 1)
    axes[row, col].grid(True, alpha=0.3)

plt.suptitle('Intensity Histograms Across Different Faces', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Edge Detection & Feature Extraction
Apply edge detection algorithms to identify important features.

In [ ]:
if CV2_AVAILABLE and SKIMAGE_AVAILABLE:
    # Convert to uint8 for OpenCV
    sample_img_uint8 = (sample_img * 255).astype(np.uint8)
    
    # Apply Canny edge detection
    edges_canny = cv2.Canny(sample_img_uint8, 50, 150)
    
    # Apply Sobel edge detection
    sobel_x = cv2.Sobel(sample_img_uint8, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(sample_img_uint8, cv2.CV_64F, 0, 1, ksize=3)
    sobel_combined = np.sqrt(sobel_x**2 + sobel_y**2)
    
    # Display results
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    axes[0, 0].imshow(sample_img, cmap='gray')
    axes[0, 0].set_title('Original Image')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(edges_canny, cmap='gray')
    axes[0, 1].set_title('Canny Edge Detection')
    axes[0, 1].axis('off')
    
    axes[1, 0].imshow(sobel_x, cmap='gray')
    axes[1, 0].set_title('Sobel X (Vertical Edges)')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(sobel_combined, cmap='gray')
    axes[1, 1].set_title('Sobel Combined')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("OpenCV and/or scikit-image not available for edge detection")

## 5. HOG (Histogram of Oriented Gradients) Features
Extract HOG features which capture edge directions and are useful for object detection.

In [ ]:
if SKIMAGE_AVAILABLE:
    # Extract HOG features and visualization
    fd, hog_image = hog(sample_img, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), visualize=True)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].imshow(sample_img, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(hog_image, cmap='gray')
    axes[1].set_title('HOG Features Visualization')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"HOG feature vector length: {len(fd)}")
    print(f"First 10 HOG features: {fd[:10]}")
else:
    print("scikit-image not available for HOG extraction")

## 6. Corner Detection
Identify key points (corners) in the images using Harris corner detector.

In [ ]:
if CV2_AVAILABLE:
    # Harris corner detection
    sample_img_uint8 = (sample_img * 255).astype(np.uint8)
    corners = cv2.cornerHarris(sample_img_uint8, blockSize=2, ksize=3, k=0.04)
    
    # Dilate for better visualization
    corners_dilated = cv2.dilate(corners, None)
    
    # Create display image
    display_img = cv2.cvtColor(sample_img_uint8, cv2.COLOR_GRAY2RGB)
    display_img[corners_dilated > 0.01 * corners_dilated.max()] = [255, 0, 0]  # Mark corners in red
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].imshow(sample_img, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(display_img)
    axes[1].set_title('Harris Corner Detection')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("OpenCV not available for corner detection")

## 7. Dimensionality Reduction Comparison
Compare PCA, t-SNE, UMAP, and Isomap for visualizing the high-dimensional face space.

In [ ]:
# Reduce to subset for faster computation
n_samples_subset = 200
X_subset = X[:n_samples_subset]
y_subset = y[:n_samples_subset]

print(f"Using {n_samples_subset} samples for dimensionality reduction comparison")

In [ ]:
# Apply all reduction methods
print("Applying PCA...")
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_subset)

print("Applying t-SNE...")
tsne = TSNE(n_components=2, random_state=42, init='pca', learning_rate='auto')
X_tsne = tsne.fit_transform(X_subset)

print("Applying Isomap...")
isomap = Isomap(n_components=2, n_neighbors=5)
X_isomap = isomap.fit_transform(X_subset)

if UMAP_AVAILABLE:
    print("Applying UMAP...")
    umap_reducer = umap.UMAP(n_components=2, random_state=42)
    X_umap = umap_reducer.fit_transform(X_subset)
else:
    print("UMAP not available")
    X_umap = None

print("All reductions complete!")

In [ ]:
# Visualize all methods
n_plots = 4 if UMAP_AVAILABLE else 3
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

# Color by first 10 people for clarity
mask = y_subset < 10

# PCA
scatter = axes[0].scatter(X_pca[mask, 0], X_pca[mask, 1], 
                         c=y_subset[mask], cmap='tab10', s=60, alpha=0.8)
axes[0].set_title('PCA', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Component 1')
axes[0].set_ylabel('Component 2')
axes[0].grid(True, alpha=0.3)

# t-SNE
axes[1].scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
               c=y_subset[mask], cmap='tab10', s=60, alpha=0.8)
axes[1].set_title('t-SNE', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Dimension 1')
axes[1].set_ylabel('Dimension 2')
axes[1].grid(True, alpha=0.3)

# Isomap
axes[2].scatter(X_isomap[mask, 0], X_isomap[mask, 1], 
               c=y_subset[mask], cmap='tab10', s=60, alpha=0.8)
axes[2].set_title('Isomap', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Dimension 1')
axes[2].set_ylabel('Dimension 2')
axes[2].grid(True, alpha=0.3)

# UMAP (if available)
if UMAP_AVAILABLE:
    axes[3].scatter(X_umap[mask, 0], X_umap[mask, 1], 
                   c=y_subset[mask], cmap='tab10', s=60, alpha=0.8)
    axes[3].set_title('UMAP', fontsize=14, fontweight='bold')
    axes[3].set_xlabel('Dimension 1')
    axes[3].set_ylabel('Dimension 2')
    axes[3].grid(True, alpha=0.3)
else:
    axes[3].text(0.5, 0.5, 'UMAP not available\nInstall: pip install umap-learn',
                ha='center', va='center', fontsize=12)
    axes[3].set_title('UMAP (Not Available)', fontsize=14)
    axes[3].axis('off')

plt.suptitle('Dimensionality Reduction Comparison (First 10 People)', 
            fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

## 8. Method Comparison Summary
Compare the strengths and characteristics of each method.

In [ ]:
print("="*60)
print("DIMENSIONALITY REDUCTION METHOD COMPARISON")
print("="*60)

print("\n1. PCA (Principal Component Analysis)")
print("   - Linear method")
print("   - Preserves global structure")
print("   - Fast and deterministic")
print(f"   - Variance explained: {pca.explained_variance_ratio_.sum():.2%}")

print("\n2. t-SNE (t-Distributed Stochastic Neighbor Embedding)")
print("   - Non-linear method")
print("   - Preserves local structure")
print("   - Good for visualization")
print("   - Can be slow and non-deterministic")

print("\n3. Isomap")
print("   - Non-linear method")
print("   - Uses geodesic distances")
print("   - Preserves global structure")
print("   - Sensitive to noise")

if UMAP_AVAILABLE:
    print("\n4. UMAP (Uniform Manifold Approximation and Projection)")
    print("   - Non-linear method")
    print("   - Preserves both local and global structure")
    print("   - Faster than t-SNE")
    print("   - More stable and reproducible")

print("\n" + "="*60)